In [ ]:
import csv
import copy

min_sup_fixed = 700
max_tree_length = 30
trTimeThresh = 15
CITY = 'NYC'

freq_patterns_path = f'/kdd_lstw_extraction/frequent_trees_City_MSF-{min_sup_fixed}_MTL-{max_tree_length}_TTR-{trTimeThresh}_{CITY}.csv'
output_path = f'/kdd_lstw_extraction/frequent_trees_City_MSF-{min_sup_fixed}_MTL-{max_tree_length}_TTR-{trTimeThresh}_{CITY}_conf.csv'

support_dict = {}

with open(freq_patterns_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        if row[0].startswith('City'):
            continue
        support_dict[row[1]] = (float(row[2]),float(row[3]))

print('#' * 40)
print(f'CITY: {CITY}\nMinimum Support Fixed: {min_sup_fixed}\nMax Tree Length: {max_tree_length}')
print('#' * 40)
print('') 


def get_patterns(p): 
    parts = p.split(' ➜ ')
    if len(parts) == 1:
        return [p]
    last_tree = copy.copy(parts)
    
    patterns_tmp = []
    patterns = []
    del last_tree[len(last_tree)-1]
    while last_tree[len(last_tree)-1] == '-1':
        del last_tree[len(last_tree)-1]
    
    patterns_tmp.append(last_tree)
    
    if '-1' in parts:
        for i,l in enumerate(parts):
            if l != '-1':
                continue
            else:
                if parts[i-1] != '-1':
                    new_pattern = copy.copy(parts)
                    del new_pattern[i-1:i+1]
                    patterns_tmp.append(new_pattern)
    
    for p in patterns_tmp:
        pattern = ''
        for _p in p:
            pattern += _p
            pattern += ' ➜ '
        patterns.append(pattern[:-3])
    
    return patterns

w = open(output_path, 'w')
w.write('City,Pattern,Pattern2,Support,SupportABS,Confidence\n')
w.close()

w = open(output_path, 'a')
for p in support_dict:
    parts = p.split(' ➜ ')
    patterns = get_patterns(p)
    if '-1' in p:
        patterns = set(patterns)
        sp = support_dict[p]
        for _p in patterns:
            if _p in support_dict:
                sp = support_dict[_p]
                w.write(CITY + ',' + p + ',' + _p + ',' + str(support_dict[p][0]) + ',' + str(support_dict[p][1]) + ',' + str(round(support_dict[p][0]/sp[0],3)) + '\n')
            else:
                print(f'\t{_p}\t\tThis is not a frequent pattern')
    elif len(parts)==1:
        sp = support_dict[p]
        w.write(CITY + ',' + p + ',' + '-' + ',' + str(sp[0]) + ',' + str(sp[1]) + ',' + '-' + '\n')
    elif len(parts)>1:
        sp = support_dict[p]
        w.write(CITY + ',' + p + ',' + patterns[0] + ',' + str(sp[0]) + ',' + str(sp[1]) + ',' + str(round(sp[0]/support_dict[patterns[0]][0],3)) + '\n')
    
w.close()
    
